https://www.appsloveworld.com/opencv/100/29/how-to-do-remove-jagged-edges-of-an-object-in-an-image-with-python-and-opencv

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

# Load the image
im = cv2.imread('image1.jpeg')
bk = im.copy()

# Fill background with black color
cv2.floodFill(im, None, seedPoint=(1, 1), newVal=(0, 0, 0), loDiff=(5, 5, 5), upDiff=(5, 5, 5))
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

# Use "open" morphological operation for removing small contours (noise)
thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

# Background
bk[(thresh_gray > 0)] = 0
bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

# Foreground
fg = im.copy()
tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
fg_gray = cv2.cvtColor(fg, cv2.COLOR_RGB2GRAY)
fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

# Find contours (there is only one contour)
contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

# Smooth contour
x, y = contours[0].T
x = x.tolist()[0]
y = y.tolist()[0]
tck, u = splprep([x, y], u=None, s=1.0, per=1)
u_new = np.linspace(u.min(), u.max(), 150)
x_new, y_new = splev(u_new, tck, der=0)
res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
smoothened = np.asarray(res_array, dtype=np.int32)

# Build a mask
mask = np.zeros_like(thresh_gray)
cv2.drawContours(mask, [smoothened], -1, 255, -1)

# For testing
test_im = cv2.cvtColor(thresh_gray, cv2.COLOR_GRAY2RGB)
cv2.drawContours(test_im, [smoothened], 0, (0, 255, 0), 1)

# Overlay smoothed contour on the original image
res = bk.copy()
res[(mask > 0)] = fg[(mask > 0)]

# Display the images side by side using matplotlib
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image")

axes[1].imshow(cv2.cvtColor(test_im, cv2.COLOR_BGR2RGB))
axes[1].set_title("Smoothed Contour")

axes[2].imshow(cv2.cvtColor(bk, cv2.COLOR_BGR2RGB))
axes[2].set_title("Background Mask")

axes[3].imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
axes[3].set_title("Result")

plt.show()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

def smooth_contour_and_overlay(original_image):
    bk = original_image.copy()

    # Fill background with black color
    cv2.floodFill(original_image, None, seedPoint=(1, 1), newVal=(0, 0, 0), loDiff=(5, 5, 5), upDiff=(5, 5, 5))
    gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)

    ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

    # Use "open" morphological operation for removing small contours (noise)
    thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Background
    bk[(thresh_gray > 0)] = 0
    bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

    # Foreground
    fg = original_image.copy()
    tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
    fg_gray = cv2.cvtColor(fg, cv2.COLOR_RGB2GRAY)
    fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

    # Find contours (there is only one contour)
    contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

    # Smooth contour
    x, y = contours[0].T
    x = x.tolist()[0]
    y = y.tolist()[0]
    tck, u = splprep([x, y], u=None, s=1.0, per=1)
    u_new = np.linspace(u.min(), u.max(), 150)
    x_new, y_new = splev(u_new, tck, der=0)
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
    smoothened = np.asarray(res_array, dtype=np.int32)

    # Build a mask
    mask = np.zeros_like(thresh_gray)
    cv2.drawContours(mask, [smoothened], -1, 255, -1)

    # Overlay smoothed contour on the original image
    result_image = bk.copy()
    result_image[(mask > 0)] = fg[(mask > 0)]

    return result_image

# Define the path to the folder containing the images
folder_path = "path/to/images_folder"

# Get a list of all image file names in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image and display the results using matplotlib
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    original_image = cv2.imread(image_path)
    result_image = smooth_contour_and_overlay(original_image)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")

    axes[1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Result")

    plt.show()
